In [1]:
from target import QTargetIMQ

import numpy as np

In [2]:
# Test manual IMQ class
x_test = np.array([0., 3.])

mean = np.array([1., 2.])
cov = np.array([[2., 3.], [3., 5.]])

log_p = lambda x:-np.log(2*np.pi) - 0.5*np.log(np.linalg.det(cov)) - 0.5*(x-mean)@np.linalg.inv(cov)@(x-mean)
grad_log_p = lambda x: -np.linalg.inv(cov)@(x-mean)
hess_log_p = lambda x: -np.linalg.inv(cov)
linv = np.linalg.inv(cov)

test_manual_imq = QTargetIMQ(log_pi=log_p, grad_log_pi=grad_log_p, hess_log_pi=hess_log_p, linv=linv)

print("log-pdf   :", test_manual_imq.log_pi(x_test))
print("glog-pdf  :", test_manual_imq.grad_log_pi(x_test))
print("hlog-pdf  :", test_manual_imq.hess_log_pi(x_test))
print("log-q-pdf :", test_manual_imq.log_q(x_test))
print("glog-q-pdf:", test_manual_imq.grad_log_q(x_test))

log-pdf   : -8.337877066409348
glog-pdf  : [ 8. -5.]
hlog-pdf  : [[-5.  3.]
 [ 3. -2.]]
log-q-pdf : -6.05570297067543
glog-q-pdf: [ 7.42708333 -4.64583333]
